In [ ]:
# Project imports
from src.get_data import (
    fetch_noaa_ca_station_metadata,
    fallback_ca_station_list,
    fetch_water_level_timeseries
)

from src.clean_data import (
    attach_housing_data,
    save_processed_dataset
)

from src.run_analysis import (
    build_trend_table,
    compute_risk_score,
    save_trend_table,
    save_risk_table
)

from src.visualize_results import (
    run_all_visualizations
)

import os
import time
import pandas as pd

In [ ]:
stations_df = fetch_noaa_ca_station_metadata()

if stations_df.empty:
    stations_df = fallback_ca_station_list()

stations_df.head()

In [ ]:
timeseries_dict = {}

for _, row in stations_df.iterrows():
    station_id = row["station_id"]
    print("Fetching water level data for:", station_id)

    ts = fetch_water_level_timeseries(
        station_id,
        days_back=DAYS_BACK
    )

    time.sleep(SLEEP_BETWEEN_STATIONS_SEC)

    if ts is not None and not ts.empty:
        timeseries_dict[station_id] = ts

len(timeseries_dict)


In [ ]:
trend_df = build_trend_table(
    stations_df=stations_df,
    timeseries_dict=timeseries_dict,
    days_back=DAYS_BACK
)

trend_path = save_trend_table(trend_df)

In [ ]:
trend_with_housing = attach_housing_data(trend_df)

processed_path = save_processed_dataset(
    trend_with_housing,
    filename="trend_with_housing.csv"
)

trend_with_housing.head()

In [ ]:
risk_df = compute_risk_score(trend_with_housing)

risk_path = save_risk_table(risk_df)
risk_df.head()

In [ ]:
outputs = run_all_visualizations()
outputs